In [ ]:
%pip install spark

In [ ]:
%pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.master('local[*]').appName('transformacao com Spark 1').getOrCreate()

In [ ]:
spark

In [ ]:
dados = spark.read\
    .format('csv')\
    .option('inferSchema', 'true')\
    .option('sep', ',')\
    .option('header', 'true')\
    .load('dados/dados_clientes.csv')

### Explorando dados

In [ ]:
dados

In [ ]:
dados.limit(15).toPandas()

In [ ]:
dados.count()

In [ ]:
dados.groupBy('Churn').count().show()

In [ ]:
dados.printSchema()

### Colunas binárias

In [ ]:
colunasBinarias = [
    'Churn',
    'Conjuge',
    'Dependentes',
    'TelefoneFixo',
    'MaisDeUmaLinhaTelefonica',
    'SegurancaOnline',
    'BackupOnline',
    'SeguroDispositivo',
    'SuporteTecnico',
    'TVaCabo',
    'StreamingFilmes',
    'ContaCorreio'
]

In [ ]:
from pyspark.sql import functions as f

In [ ]:
todasColunas = [f.when(f.col(c)== 'Sim', 1).otherwise(0).alias(c) for c in colunasBinarias] 

In [ ]:
todasColunas
# Podemos ler a primeira linha como: quando Churn for igual a "Sim", 
# seu valor será 1; caso contrário, será igual a 0. "AS Churn" é a atribuição do apelido para a coluna.

In [ ]:
for coluna in reversed(dados.columns): 
    if coluna not in colunasBinarias:
        todasColunas.insert(0, coluna)
todasColunas

In [ ]:
dados.select(todasColunas).show()

In [ ]:
dataset = dados.select(todasColunas)

In [ ]:
dataset.printSchema()

### Criando dummies

In [ ]:
dados.select(['Internet', 'TipoContrato', 'MetodoPagamento']).show()

In [ ]:
dataset.groupBy('id').pivot('Internet').agg(f.lit(1)).na.fill(0).show()

In [ ]:
Internet = dataset.groupBy('id').pivot('Internet').agg(f.lit(1)).na.fill(0)
TipoContrato = dataset.groupBy('id').pivot('TipoContrato').agg(f.lit(1)).na.fill(0)
MetodoPagamento = dataset.groupBy('id').pivot('MetodoPagamento').agg(f.lit(1)).na.fill(0)

In [ ]:
dataset = dataset\
    .join(Internet, 'id', how='inner')\
    .join(TipoContrato, 'id', how='inner')\
    .join(MetodoPagamento, 'id', how='inner')\
    .select(
        '*',
        f.col('DSL').alias('Internet_DSL'), 
        f.col('FibraOptica').alias('Internet_FibraOptica'), 
        f.col('Nao').alias('Internet_Nao'), 
        f.col('Mensalmente').alias('TipoContrato_Mensalmente'), 
        f.col('UmAno').alias('TipoContrato_UmAno'), 
        f.col('DoisAnos').alias('TipoContrato_DoisAnos'), 
        f.col('DebitoEmConta').alias('MetodoPagamento_DebitoEmConta'), 
        f.col('CartaoCredito').alias('MetodoPagamento_CartaoCredito'), 
        f.col('BoletoEletronico').alias('MetodoPagamento_BoletoEletronico'), 
        f.col('Boleto').alias('MetodoPagamento_Boleto')        
    ).drop(
        'Internet', 'TipoContrato', 'MetodoPagamento', 'DSL', 
        'FibraOptica', 'Nao', 'Mensalmente', 'UmAno', 'DoisAnos', 
        'DebitoEmConta', 'CartaoCredito', 'BoletoEletronico', 'Boleto'
    )

In [ ]:
dataset.printSchema()

### Salvando

In [ ]:
dataset.write.mode("overwrite").format('csv').save('/dados/')